# Scavenger hunt
___

Now it's your turn! Here are the questions I would like you to get the data to answer:

* Which hours of the day do the most accidents occur during?
    * Return a table that has information on how many accidents occurred in each hour of the day in 2015, sorted by the the number of accidents which occurred each hour. Use either the accident_2015 or accident_2016 table for this, and the timestamp_of_crash column. (Yes, there is an hour_of_crash column, but if you use that one you won't get a chance to practice with dates. :P)
    * **Hint:** You will probably want to use the [EXTRACT() function](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#extract_1) for this.
* Which state has the most hit and runs?
    * Return a table with the number of vehicles registered in each state that were involved in hit-and-run accidents, sorted by the number of hit and runs. Use either the vehicle_2015 or vehicle_2016 table for this, especially the registration_state_name and hit_and_run columns.

In order to answer these questions, you can fork this notebook by hitting the blue "Fork Notebook" at the very top of this page (you may have to scroll up). "Forking" something is making a copy of it that you can edit on your own without changing the original.

In [10]:
# Your code goes here :)
import bq_helper
accidents = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="nhtsa_traffic_fatalities")
accidents.list_tables()[:5]

['accident_2015', 'accident_2016', 'cevent_2015', 'cevent_2016', 'damage_2015']

In [11]:
accidents.table_schema('accident_2015')

[SchemaField('state_number', 'integer', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC). For more info on the codes, please look at <C1/V1/D1/PC1/P1/NM1 State Number> section in the pdf: https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812315', ()),
 SchemaField('state_name', 'string', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC).', ()),
 SchemaField('consecutive_number', 'integer', 'NULLABLE', 'This data element is the unique case number assigned to each crash. It appears on each data file and is used to merge information from the data files together. xxxxxx Two Characters for State Code followed by Four Characters for Case Number', ()),
 SchemaField('number_of_vehicle_f

In [13]:
query1 = '''
SELECT COUNT(consecutive_number) as counts, EXTRACT(HOUR FROM timestamp_of_crash) as hours
FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
GROUP BY hours
ORDER BY counts DESC
'''
#check query size
accidents.estimate_query_size(query1)

0.0004848688840866089

In [16]:
accidents_by_hour = accidents.query_to_pandas(query1)
accidents_by_hour.head()

,counts,hours
0,1895,18
1,1862,20
2,1838,17
3,1811,21
4,1784,19


In [22]:
accidents.table_schema('vehicle_2015')

[SchemaField('state_number', 'integer', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC). For more info on the codes, please look at <C1/V1/D1/PC1/P1/NM1 State Number> section in the pdf: https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812315', ()),
 SchemaField('consecutive_number', 'integer', 'NULLABLE', 'This data element is the unique case number assigned to each crash. It appears on each data file and is used to merge information from the data files together. xxxxxx Two Characters for State Code followed by Four Characters for Case Number', ()),
 SchemaField('vehicle_number', 'integer', 'NULLABLE', 'This data element is the consecutive number assigned to each vehicle in the case. This data element appears on each vehicle level data file and is used in conjunction with the ST_CASE data element to merge information from v

In [ ]:
test = """
SELECT consecutive_number, registration_state_name, hit_and_run
FROM `bigquery-public-data.nhtsa_traffic_fatalities.vehicle_2015`
"""
test_table = accidents.query_to_pandas_safe(test,max_gb_scanned=0.01)

In [25]:
test_table.hit_and_run.value_counts()

No         47629
Yes         1809
Unknown       39
Name: hit_and_run, dtype: int64

In [35]:
query2 = """
SELECT states, Count(id) as hit_and_run_counts
FROM (
    SELECT registration_state_name as states, consecutive_number as id, hit_and_run
    FROM `bigquery-public-data.nhtsa_traffic_fatalities.vehicle_2015`
    WHERE hit_and_run = 'Yes'
    ) as SubQuery
GROUP BY states
ORDER BY hit_and_run_counts DESC
"""
hit_and_run_counts_by_states = accidents.query_to_pandas(query2)

In [36]:
hit_and_run_counts_by_states.head()

,states,hit_and_run_counts
0,Unknown,834
1,California,155
2,Florida,96
3,Texas,86
4,New York,38


Please feel free to ask any questions you have in this notebook or in the [Q&A forums](https://www.kaggle.com/questions-and-answers)! 

Also, if you want to share or get comments on your kernel, remember you need to make it public first! You can change the visibility of your kernel under the "Settings" tab, on the right half of your screen.